In [1]:
import pandas as pd
import numpy as np
from env import username, password, host
import wrangle
import prepare
from scipy import stats
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Acquire

In [3]:
url = f"mysql+pymysql://{username}:{password}@{host}/zillow"
# this query brings in all the columns necessary to ask the questions posed by the zillow team  
query = """
            
SELECT parcelid, bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, taxamount, assessmentyear, regionidcounty,regionidzip, transactiondate
FROM properties_2017

LEFT JOIN propertylandusetype USING(propertylandusetypeid)

JOIN predictions_2017 USING(parcelid)

WHERE propertylandusedesc IN ("Single Family Residential",                       
                              "Inferred Single Family Residential")
                              AND (transactiondate BETWEEN '2017-05-01' AND '2017-08-31');
                              
                              """
    
df = pd.read_sql(query, url)
df

,parcelid,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,taxamount,assessmentyear,regionidcounty,regionidzip,transactiondate
0,14170975,4.0,2.5,2065.0,371009.0,4584.98,2016.0,1286.0,97068.0,2017-05-08
1,12354372,2.0,1.0,756.0,83520.0,1355.47,2016.0,3101.0,96119.0,2017-05-08
2,13973867,4.0,3.0,1850.0,517000.0,6116.96,2016.0,1286.0,97004.0,2017-05-08
3,13981618,3.0,2.5,3271.0,1814378.0,19686.04,2016.0,1286.0,96978.0,2017-05-08
4,17120696,4.0,2.5,1807.0,265679.0,3150.70,2016.0,2061.0,97107.0,2017-05-08
...,...,...,...,...,...,...,...,...,...,...
28119,11290535,4.0,2.0,1784.0,114569.0,2079.84,2016.0,3101.0,97318.0,2017-08-30
28120,11927617,3.0,2.0,1345.0,634832.0,7395.36,2016.0,3101.0,96517.0,2017-08-30
28121,14210331,2.0,1.5,1087.0,319000.0,3570.20,2016.0,1286.0,97063.0,2017-08-30
28122,11446352,4.0,2.0,1271.0,514037.0,5996.69,2016.0,3101.0,96162.0,2017-08-30


In [4]:
### PREARE ###

In [5]:
# I need to prepare one df for answering the questions posed and a 2nd df name df2 that is for exploring and modeling

In [6]:
df.info()
# these are all the correct datatype

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28124 entries, 0 to 28123
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      28124 non-null  int64  
 1   bedroomcnt                    28124 non-null  float64
 2   bathroomcnt                   28124 non-null  float64
 3   calculatedfinishedsquarefeet  28077 non-null  float64
 4   taxvaluedollarcnt             28123 non-null  float64
 5   taxamount                     28123 non-null  float64
 6   assessmentyear                28124 non-null  float64
 7   regionidcounty                28124 non-null  float64
 8   regionidzip                   28107 non-null  float64
 9   transactiondate               28124 non-null  object 
dtypes: float64(8), int64(1), object(1)
memory usage: 2.1+ MB


In [7]:
# here I check out all the columns I brought in to make sure I have all
# of the information necessary to answer questions posed by the team
df.head()

,parcelid,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,taxamount,assessmentyear,regionidcounty,regionidzip,transactiondate
0,14170975,4.0,2.5,2065.0,371009.0,4584.98,2016.0,1286.0,97068.0,2017-05-08
1,12354372,2.0,1.0,756.0,83520.0,1355.47,2016.0,3101.0,96119.0,2017-05-08
2,13973867,4.0,3.0,1850.0,517000.0,6116.96,2016.0,1286.0,97004.0,2017-05-08
3,13981618,3.0,2.5,3271.0,1814378.0,19686.04,2016.0,1286.0,96978.0,2017-05-08
4,17120696,4.0,2.5,1807.0,265679.0,3150.70,2016.0,2061.0,97107.0,2017-05-08


In [8]:
# I don't like the decimal places for bedroomcnt, bathroomcnt, assessmentyear, and regionidcity 
# I want to change them to integers

In [9]:
# here I want to get an idea of the summarized numbers
df.describe()

,parcelid,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,taxamount,assessmentyear,regionidcounty,regionidzip
count,2.812400e+04,28124.000000,28124.000000,28077.000000,2.812300e+04,28123.000000,28124.0,28124.000000,28107.000000
mean,1.304239e+07,3.309487,2.316509,1934.930014,5.350049e+05,6508.829263,2016.0,2509.060909,96656.246629
std,3.747700e+06,0.943073,1.016983,995.684457,7.076811e+05,8221.720405,0.0,808.588534,5130.262567
min,1.071186e+07,0.000000,0.000000,240.000000,1.000000e+03,51.260000,2016.0,1286.000000,95982.000000
25%,1.151380e+07,3.000000,2.000000,1278.000000,1.986360e+05,2705.330000,2016.0,1286.000000,96208.000000
50%,1.259670e+07,3.000000,2.000000,1675.000000,3.827980e+05,4735.280000,2016.0,3101.000000,96414.000000
75%,1.414914e+07,4.000000,3.000000,2326.000000,6.327670e+05,7528.405000,2016.0,3101.000000,96995.000000
max,1.676878e+08,12.000000,11.000000,17245.000000,2.385837e+07,276797.830000,2016.0,3101.000000,399675.000000


In [10]:
# next I'll look at null values
df.isnull().sum()

parcelid                         0
bedroomcnt                       0
bathroomcnt                      0
calculatedfinishedsquarefeet    47
taxvaluedollarcnt                1
taxamount                        1
assessmentyear                   0
regionidcounty                   0
regionidzip                     17
transactiondate                  0
dtype: int64

In [11]:
# let's compare those to the overall numbers in the dataset to see how to handle them
df.shape

(28124, 10)

In [12]:
# the amount of null values is small enough in comparison to the overall set to delete these
# records

df = df.dropna()

In [13]:
# let's check that they are gone
df.isnull().sum()

parcelid                        0
bedroomcnt                      0
bathroomcnt                     0
calculatedfinishedsquarefeet    0
taxvaluedollarcnt               0
taxamount                       0
assessmentyear                  0
regionidcounty                  0
regionidzip                     0
transactiondate                 0
dtype: int64

In [14]:
df.shape

(28060, 10)

In [15]:
# now lets rename columns
df = df.rename(columns={
                            'parcelid': 'parcel_id',
                            'calculatedfinishedsquarefeet': 'sqft',
                            'bathroomcnt': 'baths',
                            'bedroomcnt': 'beds',
                            'taxvaluedollarcnt':'tax_value'})

In [16]:
df.head()

,parcel_id,beds,baths,sqft,tax_value,taxamount,assessmentyear,regionidcounty,regionidzip,transactiondate
0,14170975,4.0,2.5,2065.0,371009.0,4584.98,2016.0,1286.0,97068.0,2017-05-08
1,12354372,2.0,1.0,756.0,83520.0,1355.47,2016.0,3101.0,96119.0,2017-05-08
2,13973867,4.0,3.0,1850.0,517000.0,6116.96,2016.0,1286.0,97004.0,2017-05-08
3,13981618,3.0,2.5,3271.0,1814378.0,19686.04,2016.0,1286.0,96978.0,2017-05-08
4,17120696,4.0,2.5,1807.0,265679.0,3150.70,2016.0,2061.0,97107.0,2017-05-08


In [17]:
# calculate the tax rate and make a new column/feature 
df['tax_rate']= df['taxamount']/df['tax_value']

In [18]:
df.head()

,parcel_id,beds,baths,sqft,tax_value,taxamount,assessmentyear,regionidcounty,regionidzip,transactiondate,tax_rate
0,14170975,4.0,2.5,2065.0,371009.0,4584.98,2016.0,1286.0,97068.0,2017-05-08,0.012358
1,12354372,2.0,1.0,756.0,83520.0,1355.47,2016.0,3101.0,96119.0,2017-05-08,0.016229
2,13973867,4.0,3.0,1850.0,517000.0,6116.96,2016.0,1286.0,97004.0,2017-05-08,0.011832
3,13981618,3.0,2.5,3271.0,1814378.0,19686.04,2016.0,1286.0,96978.0,2017-05-08,0.010850
4,17120696,4.0,2.5,1807.0,265679.0,3150.70,2016.0,2061.0,97107.0,2017-05-08,0.011859


In [19]:
# time to check for duplicates and remove them
# dropping ALL duplicate values
df.drop_duplicates(subset ="parcel_id",
                     keep = False, inplace = True)
 

In [20]:
# here we can see with shape that numerous duplicate rows were dropped
df.shape

(28017, 11)

In [21]:
#train, validate, test = split_continuous(df)
target = 'tax_value'

In [22]:
train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test = prepare.train_validate_test(df, target)

In [23]:
# now it's time to prep df2 for explore
df2 = df.copy()

In [ ]:
#remove any column leaking the target so taxamount and tax_rate
df2 = df2.drop(columns=['taxamount', 'tax_rate'])

In [25]:
df2.head()

,parcel_id,beds,baths,sqft,tax_value,assessmentyear,regionidcounty,regionidzip,transactiondate
0,14170975,4.0,2.5,2065.0,371009.0,2016.0,1286.0,97068.0,2017-05-08
1,12354372,2.0,1.0,756.0,83520.0,2016.0,3101.0,96119.0,2017-05-08
2,13973867,4.0,3.0,1850.0,517000.0,2016.0,1286.0,97004.0,2017-05-08
3,13981618,3.0,2.5,3271.0,1814378.0,2016.0,1286.0,96978.0,2017-05-08
4,17120696,4.0,2.5,1807.0,265679.0,2016.0,2061.0,97107.0,2017-05-08


In [ ]:
def remove_outlier(df):
    '''
    This function will remove values that are 3 standard deviations above or below the mean for sqft, baths, beds, and tax_value.         (Our MVP values)
    '''
    new_df = df[(np.abs(stats.zscore(df['sqft'])) < 3)]
    new_df = df[(np.abs(stats.zscore(df['baths'])) < 3)]
    new_df = df[(np.abs(stats.zscore(df['beds'])) < 3)]
    new_df = df[(np.abs(stats.zscore(df['tax_value'])) < 3)]
    return df

In [ ]:
#remove_outlier(df)

In [ ]:
# cols = ['bedroomcnt','bathroomcnt','calculatedfinishedsquarefeet','taxvaluedollarcnt','taxamount','assessmentyear']
# # impute cols using model
# imputer = SimpleImputer(strategy='most_frequent')

# imputer.fit(df[cols])

# df[cols] = imputer.transform(train[cols])
    

In [ ]:
# List of columns
cols = [col for col in df.columns if col not in ['fips', 'year_built']]

In [ ]:
# n removing outliers
# df = wrangle.remove_outliers(df, 1.5, ['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet', 'taxvaluedollarcnt', 'taxamount', 'assessmentyear', 'regionidcounty'])
# df.head()

In [ ]:
df.head()

In [ ]:
#narrow down the record to those in the "hot months" May-August 2017
# this df created so far is for answering the questions posed by team
# it will not be used to create models